In [1]:
include("util_simplex.jl")

# Problème d'optimisation linéaire avec l'algorithme du simplexe

Puisque la journée est placée sous le signe de l'optimisation, consacrons nous (enfin) à un problème linéaire :

<br/>

\begin{align}
   argmin_x & \; c\cdot x \\
\text{s.c.} & \; Ax \leqslant b
\end{align}

In [2]:
A = load(Matrix{Float64}, "dataSimplex2/A.dat");
b = load(Vector{Float64}, "dataSimplex2/b.dat");
c = load(Vector{Float64}, "dataSimplex2/c.dat");

In [3]:
sol1 = simplex(A, b, c).solution

8-element Array{Float64,1}:
 1564.4858503741316 
   85.48387096774151
  746.429393672448  
    0.0             
  976.8497016969716 
    0.0             
 1261.6013897416503 
    0.0             

## Analyse en arithmétique rationnelle

In [4]:
const Rat = Rational{BigInt}

sol2 = simplex(Rat.(A), Rat.(b), Rat.(c)).solution .|> value

8-element Array{Float64,1}:
 1564.4858503741316 
   85.48387096774151
  746.429393672448  
    0.0             
  976.8497016969716 
    0.0             
 1261.6013897416503 
    0.0             

In [5]:
sol2 .- sol1

8-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

## Analyse en arithmétique stochastique

In [6]:
function test(T)
    try
        res = simplex(T.(A), T.(b), T.(c))
        value.(res.solution)
    catch e
        value.(NaN*ones(T, length(c)))
    end
end

test (generic function with 1 method)

In [7]:
hcat(test(Rational{BigInt}), test(Float64), (test(SFloat64) for _ in 1:15)...)

8×17 Array{Float64,2}:
 1564.49    1564.49    1564.49    …  NaN  NaN  1564.49    1564.49    NaN
   85.4839    85.4839    85.4839     NaN  NaN    85.4839    85.4839  NaN
  746.429    746.429    746.429      NaN  NaN   746.429    746.429   NaN
    0.0        0.0        0.0        NaN  NaN     0.0        0.0     NaN
  976.85     976.85     976.85       NaN  NaN   976.85     976.85    NaN
    0.0        0.0        0.0     …  NaN  NaN     0.0        0.0     NaN
 1261.6     1261.6     1261.6        NaN  NaN  1261.6     1261.6     NaN
    0.0        0.0        0.0        NaN  NaN     0.0        0.0     NaN

## Analyse de la version "réparée"

In [8]:
fix_simplex = true
hcat(test(Rational{BigInt}), test(Float64), (test(SFloat64) for _ in 1:15)...)

8×17 Array{Float64,2}:
 1564.49    1564.49    1564.49    1564.49    …  1564.49    1564.49    1564.49  
   85.4839    85.4839    85.4839    85.4839       85.4839    85.4839    85.4839
  746.429    746.429    746.429    746.429       746.429    746.429    746.429 
    0.0        0.0        0.0        0.0           0.0        0.0        0.0   
  976.85     976.85     976.85     976.85        976.85     976.85     976.85  
    0.0        0.0        0.0        0.0     …     0.0        0.0        0.0   
 1261.6     1261.6     1261.6     1261.6        1261.6     1261.6     1261.6   
    0.0        0.0        0.0        0.0           0.0        0.0        0.0   

In [9]:
@reliable_digits simplex(SFloat64.(A), SFloat64.(b), SFloat64.(c)).solution

8-element Array{Tuple{Float64,Float64},1}:
 (1564.4858503741316, 15.892203692224989)
 (85.4838709677415, 15.906900233197321)  
 (746.429393672448, 15.737428125292102)  
 (0.0, NaN)                              
 (976.8497016969716, 16.02216319129637)  
 (0.0, NaN)                              
 (1261.6013897416506, 15.68171260830974) 
 (0.0, NaN)                              

In [10]:
using BenchmarkTools
@btime simplex(A, b, c)
@btime simplex(SFloat64.(A), SFloat64.(b), SFloat64.(c))
@btime simplex(Rat.(A), Rat.(b), Rat.(c));

  54.476 μs (185 allocations: 76.67 KiB)
  209.208 μs (250 allocations: 64.19 KiB)
  29.727 ms (859894 allocations: 18.27 MiB)
